In [1]:
# In this Notebook we will create custom tools in crew ai and use that to answer the queries
# The tools that we are going to create are : calculator tool, get_employee_name_from_id tool and get_employee_salary tool

In [6]:
from crewai import Agent, Task, Crew, LLM
from crewai.tools import BaseTool

In [4]:
openai_api_key = 'your-openai-api-key' # or load from .env file
model_name = 'gpt-3.5-turbo'

llm = LLM(model=model_name,api_key=openai_api_key)

In [28]:
# custom tools
# A custom tool needs name and description.

class calc_tool(BaseTool):
    
    name : str = "calc_tool"
    description : str = ("Takes the given expression and calcuates the output.")

    def _run(self,what:str):
        return eval(what)

class GetEmployeeSalary(BaseTool):
    
    name : str = "Get Employee Salary"
    description : str = ("Takes the employee name and return the salary")

    def _run(self,employee_name:str):
        if 'rahul' in employee_name.lower():
            return '40LPA'
        else:
            return '20LPA'


class GetEmployeeNameFromID(BaseTool):
    
    name : str = "Get Employee Name From ID"
    description : str = ("Takes the employee id and return the employee name")

    def _run(self,employee_id:str):
        if 'ad241' in employee_id.lower():
            return 'Rahul'
        else:
            return 'other'

In [29]:
calc_tool = calc_tool()
employee_salary_tool = GetEmployeeSalary()
employee_name_tool = GetEmployeeNameFromID()

In [30]:
query_agent = Agent(role="Lead HR",
                    goal="Answer the user question based on the tools available.",
                    backstory="You work as Lead HR in Sigmoid analytics"
                               "You have access to certain tool that would help answer the given answer"
                               "If you cant answer the question, just say i dont know the answer in a polite manner."
                               "Dont try to imagine the answer",
                    tools=[calc_tool,employee_salary_tool,employee_name_tool],
                    llm=llm,verbose=True)


In [31]:
claculating_task = Task(description="Answers the given user query : {query} using the tools available",
                       expected_output="Answer for the user query."
                        "User may give an employee name or employee id"
                        "when user gives employee id first convert that to name using tools"
                        "then try to answer the query.",
                       tools=[calc_tool,employee_salary_tool,employee_name_tool],
                       agent = query_agent)

In [32]:
crew = Crew(tasks=[claculating_task],agents=[query_agent],verbose=True)

2025-01-12 15:38:51,034 - 8471115008 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [25]:
inputs = {
    "query": "What is the average salary of Karthik and Rahul?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Lead HR
## Task: Answers the given user query : What is the average salary of Karthik and Rahul? using the tools available
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for GetEmployeeSalarySchema
employee_name
  Field required [type=missing, input_value={'employee_id': 'Karthik'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Get Employee Salary accepts these inputs: Tool Name: Get Employee Salary
Tool Arguments: {'employee_name': {'description': None, 'type': 'str'}}
Tool Description: Takes the employee name and return the salary



# Agent: Lead HR
## Thought: I need to first convert the employee names "Karthik" and "Rahul" to their respective employee IDs to find their salaries and then calculate the average salary.
## Using tool: Get Employee Salary
## Tool Input: 
"{\"employee_id\": \"Karthik\"}"
## Tool Output: 

I encountered an error whi

In [33]:
inputs = {
    "query": "What is the average salary of Karthik and employee id AD241?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Lead HR
## Task: Answers the given user query : What is the average salary of Karthik and employee id AD241? using the tools available


# Agent: Lead HR
## Thought: I need to first convert the employee ID "AD241" to the employee name using the available tools, and then find the average salary of both Karthik and the employee with ID "AD241".
## Using tool: Get Employee Name From ID
## Tool Input: 
"{\"employee_id\": \"AD241\"}"
## Tool Output: 
Rahul


# Agent: Lead HR
## Using tool: Get Employee Salary
## Tool Input: 
"{\"employee_name\": \"Karthik\"}"
## Tool Output: 
20LPA


# Agent: Lead HR
## Using tool: Get Employee Salary
## Tool Input: 
"{\"employee_name\": \"Rahul\"}"
## Tool Output: 
40LPA


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: calc_tool
Tool Arguments: {'what': {'description': None, 'type': 'str'}}
Tool Description: Takes the given expression and calcuates the output.
Tool Name: Get Employ